# Load data

In [ ]:
# Mount Google Drive locally 
# Using the instructions found here https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA&line=1&uniqifier=1

In [ ]:
# Load the pickled dataframe using the "read_pickle()"" function from "pandas"
# df = ...

## Train and run a Logistic Regression model with Reweighing

In [ ]:
# install AIF360
!pip install aif360
!pip install fairlearn

In [ ]:
# Instantiate the classifier
from sklearn.linear_model import LogisticRegression
# lr = ...

# instantiate the cross-validation scheme
from sklearn.model_selection import StratifiedKFold
# mv = ...

# setup the performance metrics to be computed
from sklearn import metrics
# perf_metrics = {"Accuracy": ..., 
#                 "Precision": ..., 
#                 "Recall": ...,
#                 "AUC": ..., 
#                 "F1-Score": ..., 
#                 }

# Set up the fairness metrics to be computed using AIF360
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# Set up reweighing
from aif360.algorithms.preprocessing import Reweighing

# privileged_group = ...
# unprivileged_group = ...

In [ ]:
# Train a logistic regression classifier on the dataset after reweighing
import numpy as np
import pandas as pd
k, i = True, 1

# instantiating X
# X = ...

# instantiating the target variable
# y = ...

for (train, test) in mv.split(X, y):

    # Reweigh the training data
    dataset = X.iloc[train].copy()
    dataset[df.columns[-1]] = y.iloc[train]
    # dataset_BLD = BinaryLabelDataset(...)
    # rw = Reweighing(...)
    dataset_transformed = rw.fit_transform(dataset_BLD)

    # instantiating X
    X_train = pd.DataFrame(data = dataset_transformed.features, columns=df.columns[:-1]) 

    # instantiating y
    y_train = pd.DataFrame(data = dataset_transformed.labels.ravel())

    # instantiating the reweighing weights
    w_train = pd.DataFrame(data = dataset_transformed.instance_weights)

    # fit model including sample weights calculated in reweighing
    # lr = lr.fit(...)
    
    # get predictions in the test set
    # ypred_prob = lr.predict_proba ... # get probabilities
    # ypred_class = lr.predict ...

    # compute performance metrics
    metrics = []
    dataset = X.iloc[test].copy()
    dataset[df.columns[-1]] = np.expand_dims(y.iloc[test], axis=1)
    # dataset = BinaryLabelDataset(...)
    dataset_pred = X.iloc[test].copy()
    dataset_pred[df.columns[-1]] = np.expand_dims(ypred_class, axis=1)
    # dataset_pred = BinaryLabelDataset(...)
    # metric_CM = ClassificationMetric(...)
    for pf in perf_metrics.keys():
        if pf in ["AUC", "Brier"]:
            # metrics += [[pf, ...]]
        else:
            # metrics += [[pf, ...]]
    
    # Compute fairness metrics
    # metrics += [['Statistical Parity Difference', ...]]
    # metrics += [['Disparate Impact', ...]]
    # metrics += [['Equal Opportunity Difference', ...]]
    # metrics += [['Average Odds Difference', ...]]
    # metrics += [['Accuracy Male', ...]]
    # metrics += [['Accuracy Female', ...]]

    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics = df_m.copy()
        k=0
    else:
        df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

#Review metrics

In [ ]:
# Display metrics

#  metrics_table = df_metrics.pivot_table(...)
metrics_table

In [ ]:
# Construct a DataFrame with allowable thresholds for plotting

fairness_df = pd.DataFrame(columns=["Metric", "Value", "Min", "Max"])
fairness_df.loc[0] = ['Disparate Impact'] + list((metrics_table[('mean', 'Value')]['Disparate Impact'], 0.8, 1.2))
fairness_df.loc[1] = ['Statistical Parity Difference'] + list((metrics_table[('mean', 'Value')]['Statistical Parity Difference'], -0.1, 0.1))
fairness_df.loc[2] = ['Average Odds Difference'] + list((metrics_table[('mean', 'Value')]['Average Odds Difference'], -0.1, 0.1))
fairness_df.loc[3] = ['Equal Opportunity Difference'] + list((metrics_table[('mean', 'Value')]['Equal Opportunity Difference'], -0.1, 0.1))

In [ ]:
# Chart the metrics

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import numpy as np

def plot_metrics(df_fair): 
  def plot_metric(metric, ylim):
    fig, ax = plt.subplots()
    ax = sns.barplot(data=metric, x="Metric", y="Value")
    plt.axhline(np.mean(metric[["Min", "Max"]].values), color='black')
    plt.ylim(*ylim)
    range_height = metric[["Max"]].values[0][0] - metric[["Min"]].values[0][0]
    ax.add_patch(patches.Rectangle((-1, metric[["Min"]].values[0][0]), 2 , range_height, facecolor="green", alpha=0.3))

  plot_metric(df_fair.iloc[[0]], (0, 2))
  plot_metric(df_fair.iloc[[1]], (-1, 1))
  plot_metric(df_fair.iloc[[2]], (-1, 1))
  plot_metric(df_fair.iloc[[3]], (-1, 1))

plot_metrics(fairness_df)